In [1]:
import requests
import pathlib
from pathlib import Path
import json
import time
import datetime
import random

api_key = 'RGAPI-7f2f15a0-e2cc-477c-80f6-dc89ea23bd47'

In [2]:
def save_match(match_id, tier_code):
    timeline_file = Path('./data/match_timeline/%s_%d.json' % (tier_code,match_id))
    info_file = Path('./data/match_info/%s_%d.json' % (tier_code,match_id))
    if timeline_file.is_file() and info_file.is_file():
        print('Match data already exist', end = '\r')
    else:  
        timeline_request = "https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/" + str(match_id) + "?api_key=" + api_key
        match_request = "https://kr.api.riotgames.com/lol/match/v4/matches/" + str(match_id) + "?api_key=" + api_key

        tl = requests.get(timeline_request)
        mts = requests.get(match_request)
        
        count = 0
        while ('status' in mts.json()) or ('status' in tl.json()):
            for i in range(10):
                print("Waiting "+ str(count+1), end = "\r")
                count = count + 1
                time.sleep(1)
            tl = requests.get(timeline_request)
            mts = requests.get(match_request)
        
        pathlib.Path('./data/match_timeline/%d_%d.json' % (tier_code,match_id)).write_bytes(tl.content)
        pathlib.Path('./data/match_info/%d_%d.json' % (tier_code,match_id)).write_bytes(mts.content)
        print("%s Saved"%match_id, end = '\r')

In [3]:
def match_list_gather(tier):
    try:
        tier_names = ['DIAMOND','PLATINUM', 'GOLD', 'SILVER', 'BRONZE']
        division_names = ['I','II','III','IV']

        division = random.randrange(0,4)
        page = random.randrange(1,100)
        random_summoner = requests.get("https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/%s/%s?page=%d&api_key=%s"%\
                            (tier_names[tier],division_names[division],page,api_key))

        random_dic = random_summoner.json()
        random_id = random.choice(random_dic)['summonerId']

        start = requests.get("https://kr.api.riotgames.com/lol/summoner/v4/summoners/%s?api_key=%s"%(random_id,api_key))
        start_acid = start.json()['accountId']

        first_match_list = requests.get("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/%s?api_key=%s"%(start_acid,api_key))
        match_list = (first_match_list.json())['matches']
        match_list = [i['gameId'] for i in match_list]
        print("%d %s %s match list loaded"%(len(match_list),tier_names[tier],division_names[division]))
        return match_list
    except:
        count = 0
        while(1):
            try:
                test = "https://kr.api.riotgames.com/lol/match/v4/matches/4198728328?api_key=" + api_key
                t = requests.get(test).json()
                if(t['status']['status_code'] == 429):
                    for i in range(10):
                        print("Waiting "+ str(count+1), end = "\r")
                        count = count + 1
                        time.sleep(1)
            except:
                break
        return match_list_gather(tier)

In [4]:
epoch = 12 #epoch 1당 약 21분 소요, data 500개 추가, 약 130MB 할당
start_time = time.time()

for tier in range(5):
    for ep in range(epoch):
        match_list = match_list_gather(tier)
        for match in match_list:
            save_match(match, tier+1)
        print("match list saved", end = '\r')
print("Done!, Processing Time : %s"% str(datetime.timedelta(seconds=time.time()-start_time)))

100 DIAMOND III match list loaded
100 DIAMOND III match list loaded
100 DIAMOND IV match list loaded
100 DIAMOND III match list loaded
100 DIAMOND IV match list loaded
100 DIAMOND II match list loaded
100 DIAMOND IV match list loaded
100 DIAMOND IV match list loaded
100 DIAMOND IV match list loaded
100 DIAMOND II match list loaded
100 DIAMOND IV match list loaded
100 DIAMOND I match list loaded
100 PLATINUM II match list loaded
100 PLATINUM IV match list loaded
100 PLATINUM IV match list loaded
100 PLATINUM II match list loaded
100 PLATINUM II match list loaded
100 PLATINUM II match list loaded
100 PLATINUM I match list loaded
100 PLATINUM IV match list loaded
100 PLATINUM III match list loaded
100 PLATINUM II match list loaded
100 PLATINUM III match list loaded
100 PLATINUM II match list loaded
100 GOLD III match list loaded
100 GOLD I match list loaded
100 GOLD IV match list loaded
100 GOLD III match list loaded
100 GOLD III match list loaded
100 GOLD I match list loaded
100 GOLD IV 